In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mooccubex-relations/course-teacher.txt
/kaggle/input/mooccubex-relations/concept-comment.txt
/kaggle/input/mooccubex-relations/user-video.json
/kaggle/input/mooccubex-relations/course-comment.txt
/kaggle/input/mooccubex-relations/concept-problem.txt
/kaggle/input/mooccubex-relations/user-reply.txt
/kaggle/input/mooccubex-relations/concept-video.txt
/kaggle/input/mooccubex-relations/course-school.txt
/kaggle/input/mooccubex-relations/user-comment.txt
/kaggle/input/mooccubex-relations/exercise-problem.txt
/kaggle/input/mooccubex-relations/concept-paper.txt
/kaggle/input/mooccubex-relations/reply-reply.txt
/kaggle/input/mooccubex-relations/course-field.json
/kaggle/input/mooccubex-relations/video_id-ccid.txt
/kaggle/input/mooccubex-relations/user-xiaomu.json
/kaggle/input/mooccubex-relations/user-problem.json
/kaggle/input/mooccubex-relations/comment-reply.txt
/kaggle/input/mooccubex-relations/concept-other.txt
/kaggle/input/mooccubex-entities/comment.json
/kaggle/input/mooc

In [2]:
column_names = ['course_id','school_id']
course_school = pd.read_csv('/kaggle/input/mooccubex-relations/course-school.txt', names=column_names,sep='\t')
column_names = ['course_id','teacher_id']
course_teacher = pd.read_csv('/kaggle/input/mooccubex-relations/course-teacher.txt', names=column_names, sep = '\t')

In [3]:
school = pd.read_json('/kaggle/input/mooccubex-entities/school.json', lines = True)
teacher = pd.read_json('/kaggle/input/mooccubex-entities/teacher.json', lines = True)

In [4]:
display(school.head())
school.describe()

,id,name,name_en,sign,about,motto
0,S_1,清华大学,Tsinghua University,thu,简称“清华”，由中华人民共和国教育部直属，中央直管副部级建制，位列“211工程”、“985工...,"自强不息,厚德载物"
1,S_2,北京大学,Peking University,PKU,北京大学（Peking University），简称“北大”，是中华人民共和国教育部直属的全...,博学、审问、慎思、明辨
2,S_3,武汉大学,Wuhan University,whu,武汉大学（Wuhan University）简称“武大”，是中华人民共和国教育部直属的综合性...,自强 弘毅 求是 拓新
3,S_4,苏州大学,Soochow University,suda,苏州大学（Soochow University），坐落于历史文化名城苏州，是教育部与江苏省人...,养天地正气，法古今完人
4,S_5,四川大学,Sichuan University,scu,四川大学（Sichuan University），简称“川大”，坐落于四川省会成都，是教育部...,


,id,name,name_en,sign,about,motto
count,429,429,429,429,429,429
unique,429,422,424,421,421,140
top,S_1,长安大学,Dalian University of Technology,hzic,消防高等专科学校，加挂应急管理部消防救援局昆明训练总队牌子，是应急管理部消防救援局直属的一所...,
freq,1,2,2,2,2,282


In [5]:
display(teacher.head())
teacher.describe()

,id,name,name_en,about,job_title,org_name
0,T_1,刘燕妮,Yanni Liu,北大哲学系毕业，清华大学马克思主义学院讲师。开设过的课程，马克思主义哲学；马克思主义原理；儒...,讲师,清华大学
1,T_2,陈怡,Yi Chen,1945年生于重庆，1967年华中工学院（现华中科技大学）发配电专业本科毕业，1982年南京...,教授,清华大学
2,T_3,程钢,Gang Cheng,程钢，《庄子哲学导读》课程负责人，清华大学历史系副教授，清华大学国家大学生文化素质教育基地副主任。,副教授,清华大学
3,T_4,谢维和,None,谢维和，博士、教授、博士生导师、教育研究院院长。研究方向：教育学原理、教育社会学、高等教育和...,教授,清华大学
4,T_5,史静寰,Jing-huan Shi,史静寰，女，清华大学教育研究院教授、博士生导师、清华大学第12届学位评定委员会副主席、教育学...,教授,清华大学


,id,name,name_en,about,job_title,org_name
count,17018,17018,9525,17018,17018,17018
unique,17018,13967,5062,12606,1406,1005
top,T_1,顾礼平,,,副教授,清华大学
freq,1,20,4142,3125,4288,1140


In [6]:
!pip install deep-translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.0 MB/s eta 0:00:00


In [7]:
from deep_translator import GoogleTranslator
from time import sleep

def translate_str_column(df, col, batch_size=10, retries=3):
    translator = GoogleTranslator(source="zh-CN", target="en")
    texts = df[col].astype(str).tolist()
    translated_texts = []

    for i in range(0, len(texts), batch_size):
        texts_batch = texts[i: i + batch_size]
        
        for attempt in range(retries):
            try:
                trans_batch = [translator.translate(text) for text in texts_batch]
                translated_texts.extend(trans_batch)
                break  # Thành công thì thoát vòng lặp
            except Exception as e:
                if attempt < retries - 1:
                    print(f"Lỗi: {e}. Thử lại lần {attempt + 1}...")
                    sleep(2)  # Đợi trước khi thử lại
                else:
                    print("Thất bại sau nhiều lần thử, giữ nguyên văn bản gốc.")
                    translated_texts.extend(texts_batch)  # Giữ nguyên nếu thất bại

    df[col + "_translated"] = translated_texts
    return df

In [8]:
# Example usage:


In [9]:
school = translate_str_column(school, 'about')
school = translate_str_column(school, 'motto')
school = translate_str_column(school, 'name')

In [10]:
teacher = translate_str_column(teacher, 'name')
teacher = translate_str_column(teacher, 'about')
teacher = translate_str_column(teacher, 'job_title')
teacher = translate_str_column(teacher, 'org_name')

Lỗi: Anna Fitzpatrick --> No translation was found using the current translator. Try another translator?. Thử lại lần 1...
Lỗi: 王红厂 --> No translation was found using the current translator. Try another translator?. Thử lại lần 1...
Lỗi: 孙万蓉 --> No translation was found using the current translator. Try another translator?. Thử lại lần 1...
Lỗi: 刘光颖 --> No translation was found using the current translator. Try another translator?. Thử lại lần 1...
Lỗi: 史雪岩 --> No translation was found using the current translator. Try another translator?. Thử lại lần 2...
Lỗi: 长江雨课堂 --> No translation was found using the current translator. Try another translator?. Thử lại lần 1...
Lỗi: 刘臻，硕士，助教，从事消防灭火救援教学工作2年，主讲课程《建筑火灾扑救》，发表多篇专业论文。 --> No translation was found using the current translator. Try another translator?. Thử lại lần 1...
Lỗi: 西安科技大学安全工程学院副教授，硕士研究生导师，安全工程博士/博士后，国家注册安全工程师，国家注册安全评价师，“引汉济渭”工程专家库成员、西安市安监局专家库成员，AEIC专家，主持国家自然智能科学基金、中国博士后面上项目等纵横向课题十余项，发表论文30余篇，参编著作1部，专利11项，主要为学生讲授《安全科学原理》、《安全人机工程》以

In [11]:
school

,id,name,name_en,sign,about,motto,about_translated,motto_translated,name_translated
0,S_1,清华大学,Tsinghua University,thu,简称“清华”，由中华人民共和国教育部直属，中央直管副部级建制，位列“211工程”、“985工...,"自强不息,厚德载物","It is referred to as ""Tsinghua"" for short. It ...",Self-improvement and virtue carry things,Tsinghua University
1,S_2,北京大学,Peking University,PKU,北京大学（Peking University），简称“北大”，是中华人民共和国教育部直属的全...,博学、审问、慎思、明辨,"Peking University, referred to as ""Peking Univ...","Learn, interrogate, think carefully, and disti...",Beijing University
2,S_3,武汉大学,Wuhan University,whu,武汉大学（Wuhan University）简称“武大”，是中华人民共和国教育部直属的综合性...,自强 弘毅 求是 拓新,"Wuhan University (Wuhan University, abbreviate...","Self-reliance, Hongyi, Seek truth, and expand new",Wuhan University
3,S_4,苏州大学,Soochow University,suda,苏州大学（Soochow University），坐落于历史文化名城苏州，是教育部与江苏省人...,养天地正气，法古今完人,"Soochow University is located in Suzhou, a fam...",To nourish the righteousness of heaven and ear...,Suzhou University
4,S_5,四川大学,Sichuan University,scu,四川大学（Sichuan University），简称“川大”，坐落于四川省会成都，是教育部...,,"Sichuan University (Sichuan University), refer...",,Sichuan University
...,...,...,...,...,...,...,...,...,...
424,S_1402,高雄医学大学,Kaohsiung Medical University,Kaohsiung Medical University,高雄医学大学（Kaohsiung Medical University）源自1954年创校的...,,Kaohsiung Medical University is a selfless don...,,Kaohsiung Medical University
425,S_1405,黑龙江农垦职业学院,Heilongjiang Agricultural ReclamationVocationa...,nkzy,黑龙江农垦职业学院（Heilongjiang Agricultural Reclamatio...,,Heilongjiang Agricultural Reclamation Vocation...,,Heilongjiang Agricultural Reclamation Vocation...
426,S_1448,江西财经大学,Jiangxi University of Finance And Economics,jxufe,江西财经大学（Jiangxi University of Finance and Econo...,,Jiangxi University of Finance and Economics Ec...,,Jiangxi University of Finance and Economics
427,S_1453,贵州交通职业技术学院,GuiZhou Communications Polytechnic,gzjtzy,贵州交通职业技术学院是一所以交通为特色的理工类高职院校。创办于1958年，走过国家“示范校”...,知行合一 德技双馨,Guizhou Transportation Vocational and Technica...,"Integration of knowledge and action, both virt...",Guizhou Transportation Vocational and Technica...


In [12]:
teacher

,id,name,name_en,about,job_title,org_name,name_translated,about_translated,job_title_translated,org_name_translated
0,T_1,刘燕妮,Yanni Liu,北大哲学系毕业，清华大学马克思主义学院讲师。开设过的课程，马克思主义哲学；马克思主义原理；儒...,讲师,清华大学,Liu Yanni,Graduated from the Department of Philosophy at...,lecturer,Tsinghua University
1,T_2,陈怡,Yi Chen,1945年生于重庆，1967年华中工学院（现华中科技大学）发配电专业本科毕业，1982年南京...,教授,清华大学,Chen Yi,"Born in Chongqing in 1945, he graduated from t...",professor,Tsinghua University
2,T_3,程钢,Gang Cheng,程钢，《庄子哲学导读》课程负责人，清华大学历史系副教授，清华大学国家大学生文化素质教育基地副主任。,副教授,清华大学,Cheng Gang,"Cheng Gang, the director of the course ""Introd...",Associate Professor,Tsinghua University
3,T_4,谢维和,None,谢维和，博士、教授、博士生导师、教育研究院院长。研究方向：教育学原理、教育社会学、高等教育和...,教授,清华大学,Xie Weihe,"Xie Weihe is a doctor, professor, doctoral sup...",professor,Tsinghua University
4,T_5,史静寰,Jing-huan Shi,史静寰，女，清华大学教育研究院教授、博士生导师、清华大学第12届学位评定委员会副主席、教育学...,教授,清华大学,Shi Jinghuan,"Shi Jinghuan, female, professor and doctoral s...",professor,Tsinghua University
...,...,...,...,...,...,...,...,...,...,...
17013,T_17153,邹 望,,"长春汽车工业高等专科学校2004届毕业生,从事BMW品牌售后服务管理多年,现在自己创业多年,...",,长春汽车工业高等专科学校,Zou Wang,A graduate of Changchun Automobile Industry Co...,,Changchun Automobile Industry College
17014,T_17154,王建新,,王建新，男，法学博士，中国人民公安大学治安学院副教授，硕士研究生导师，北京市优质本科课程主讲...,副教授,中国人民公安大学,Wang Jianxin,"Wang Jianxin, male, PhD in Law, associate prof...",Associate Professor,People's Public Security University of China
17015,T_17155,刘晓栋,,刘晓栋，男，博士，硕士生导师，中国人民公安大学治安学院讲师，研究方向为公共安全与应急管理、大...,讲师,中国人民公安大学,Liu Xiaodong,"Liu Xiaodong, male, doctoral, master's supervi...",lecturer,People's Public Security University of China
17016,T_17156,何钊,hezhao,《云南特色民间工艺》课程主讲教师。\n云南艺术学院设计学院产品设计专业教师，档案管理员。\n...,助教,云南艺术学院,He Zhao,"The main lecturer of the course ""Yunnan Charac...",Teaching assistant,Yunnan Academy of Arts


In [13]:
school[['id','name_translated','about_translated','motto_translated']].to_csv("en-school.csv")
teacher[['id','name_translated','about_translated','job_title_translated','org_name_translated']].to_csv("en-teacher.csv")

In [14]:
course_school.head()

,course_id,school_id
0,C_375629,S_1
1,C_375775,S_1
2,C_375778,S_1
3,C_584313,S_1
4,C_584329,S_1


In [15]:
course_school.describe()

,course_id,school_id
count,3983,3983
unique,3951,494
top,C_681512,S_1
freq,2,551


In [16]:
course_teacher.describe()

,course_id,teacher_id
count,97192,97192
unique,13866,12252
top,C_762789,T_82
freq,648,336


In [17]:
course_teacher.head()

,course_id,teacher_id
0,C_323899,T_5092
1,C_323899,T_5092
2,C_324384,T_6164
3,C_324384,T_6472
4,C_324384,T_6471
